Download the List_of_postal_codes_of_Canada to an html file:

In [3]:
!curl https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M > postal_codes.html
print('Data downloaded!')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 78670  100 78670    0     0  44122      0  0:00:01  0:00:01 --:--:-- 44122
Data downloaded!


parse the html file using BeautifulSoup:

In [5]:
from bs4 import BeautifulSoup

with open('postal_codes.html') as html_file:
    html_data = html_file.read()

    soup=BeautifulSoup(html_data,"lxml")
    #print(soup.prettify())
    result = soup.find("table")
    

In [6]:
str(result)[0:1000]

'<table class="wikitable sortable">\n<tbody><tr>\n<th>Postcode</th>\n<th>Borough</th>\n<th>Neighbourhood\n</th></tr>\n<tr>\n<td>M1A</td>\n<td>Not assigned</td>\n<td>Not assigned\n</td></tr>\n<tr>\n<td>M2A</td>\n<td>Not assigned</td>\n<td>Not assigned\n</td></tr>\n<tr>\n<td>M3A</td>\n<td><a href="/wiki/North_York" title="North York">North York</a></td>\n<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>\n</td></tr>\n<tr>\n<td>M4A</td>\n<td><a href="/wiki/North_York" title="North York">North York</a></td>\n<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>\n</td></tr>\n<tr>\n<td>M5A</td>\n<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>\n<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>\n</td></tr>\n<tr>\n<td>M6A</td>\n<td><a href="/wiki/North_York" title="North York">North York</a></td>\n<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>\n</td></tr>\n<tr>\

Transform the parse result to an object list(with properties: Postcode, Borough & Neighbourhood)

In [7]:
data_list = []
for idx, tr in enumerate(result.find_all('tr')):
    if idx != 0:
        tds = tr.find_all('td')
        data_list.append({
            'PostalCode': tds[0].contents[0],
            'Borough': tds[1].contents[0],
            'Neighbourhood': tds[2].contents[0],
        })
print(data_list[0:10])

[{'PostalCode': 'M1A', 'Borough': 'Not assigned', 'Neighbourhood': 'Not assigned\n'}, {'PostalCode': 'M2A', 'Borough': 'Not assigned', 'Neighbourhood': 'Not assigned\n'}, {'PostalCode': 'M3A', 'Borough': <a href="/wiki/North_York" title="North York">North York</a>, 'Neighbourhood': <a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>}, {'PostalCode': 'M4A', 'Borough': <a href="/wiki/North_York" title="North York">North York</a>, 'Neighbourhood': <a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>}, {'PostalCode': 'M5A', 'Borough': <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a>, 'Neighbourhood': <a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>}, {'PostalCode': 'M6A', 'Borough': <a href="/wiki/North_York" title="North York">North York</a>, 'Neighbourhood': <a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>}, {'PostalCode': 'M6A', 'Borough': <a href="/wiki/North_York" title="North Y

create the data_frame:

In [8]:
import pandas as pd
data_frame = pd.DataFrame(data_list)

In [9]:
data_frame.head(10)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,[North York],[Parkwoods]
3,M4A,[North York],[Victoria Village]
4,M5A,[Downtown Toronto],[Harbourfront]
5,M6A,[North York],[Lawrence Heights]
6,M6A,[North York],[Lawrence Manor]
7,M7A,[Queen's Park],Not assigned\n
8,M8A,Not assigned,Not assigned\n
9,M9A,[Downtown Toronto],[Queen's Park]


Define a function to process the Borough & Neighbourhood data:
    1. clear all the '\n's in the text;
    2. extract title from the <a href structrue;
    3. return the title.

In [10]:
def clean_data(x):
    result = str(x)
    result = result.replace('\n','')
    result = result.replace(',','')
    if('Not assigned' in result):
        return result
    if('href' not in result):
        return result
    #print(x)
    return re.findall('title="(.+?)"', result)[0]

Apply the function to 'Borough' and 'Neighbourhood' columns, and drop all rows where borough is Not assigned:

In [12]:
import re
data_frame['Neighbourhood'] = data_frame['Neighbourhood'].apply(lambda x:clean_data(x))
data_frame['Borough'] = data_frame['Borough'].apply(lambda x:clean_data(x))

data_frame = data_frame[data_frame['Borough']!='Not assigned'].reset_index(drop=True)

replace Not-assigned Neighbourhood with the value of Borough

In [13]:
for index, row in data_frame.iterrows():
    if(row['Neighbourhood'] == 'Not assigned'):
        row['Neighbourhood'] = row['Borough']

In [14]:
data_frame.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park (Toronto),Queen's Park (Toronto)
6,M9A,Downtown Toronto,Queen's Park (Toronto)
7,M1B,Scarborough Toronto,Rouge Toronto
8,M1B,Scarborough Toronto,Malvern Toronto
9,M3B,North York,Don Mills North


Finally we get the postalcode data:

In [15]:
data_frame.shape

(210, 3)

Group the data by ['PostalCode','Borough'] and join the Beighbourhood with comma.

In [16]:
data_frame = data_frame.groupby(['PostalCode','Borough'])['Neighbourhood'].agg(lambda x:','.join(x)).reset_index()

In [17]:
data_frame.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough Toronto,"Rouge Toronto,Malvern Toronto"
1,M1C,Scarborough Toronto,"Highland Creek (Toronto),Rouge Hill,Port Union..."
2,M1E,Scarborough Toronto,"Guildwood,Morningside Toronto,West Hill Toronto"
3,M1G,Scarborough Toronto,Woburn Toronto
4,M1H,Scarborough Toronto,Cedarbrae


In [18]:
data_frame.shape

(103, 3)

In [19]:
data_frame[data_frame['PostalCode']=='M5G']

,PostalCode,Borough,Neighbourhood
57,M5G,Downtown Toronto,Central Bay Street


Get the geospatial_coordinates data from the csv file:

In [20]:
geospatial_coordinates = pd.read_csv("./Geospatial_Coordinates.csv")

In [21]:
geospatial_coordinates

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Join dataframe according to the PostalCode:

In [22]:
result_data = pd.merge(left=data_frame, right=geospatial_coordinates, how='left', left_on='PostalCode', right_on='Postal Code')


In [23]:
result_data = result_data.drop(['Postal Code'], axis = 1)

The result is :

In [24]:
result_data

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough Toronto,"Rouge Toronto,Malvern Toronto",43.806686,-79.194353
1,M1C,Scarborough Toronto,"Highland Creek (Toronto),Rouge Hill,Port Union...",43.784535,-79.160497
2,M1E,Scarborough Toronto,"Guildwood,Morningside Toronto,West Hill Toronto",43.763573,-79.188711
3,M1G,Scarborough Toronto,Woburn Toronto,43.770992,-79.216917
4,M1H,Scarborough Toronto,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York Toronto,Weston Toronto,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Mou...",43.739416,-79.588437


In [25]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [26]:
neighborhoods = result_data[['Borough','Neighbourhood','Latitude','Longitude']]

In [27]:
neighborhoods

,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough Toronto,"Rouge Toronto,Malvern Toronto",43.806686,-79.194353
1,Scarborough Toronto,"Highland Creek (Toronto),Rouge Hill,Port Union...",43.784535,-79.160497
2,Scarborough Toronto,"Guildwood,Morningside Toronto,West Hill Toronto",43.763573,-79.188711
3,Scarborough Toronto,Woburn Toronto,43.770992,-79.216917
4,Scarborough Toronto,Cedarbrae,43.773136,-79.239476
5,Scarborough Toronto,Scarborough Village,43.744734,-79.239476
6,Scarborough Toronto,"East Birchmount Park,Ionview,Kennedy Park Toronto",43.727929,-79.262029
7,Scarborough Toronto,"Clairlea,Golden Mile Toronto,Oakridge Toronto",43.711112,-79.284577
8,Scarborough Toronto,"Cliffcrest,Cliffside Toronto,Scarborough Villa...",43.716316,-79.239476
9,Scarborough Toronto,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [28]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 12 boroughs and 103 neighborhoods.


In [29]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tnt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [109]:
# create map of Toronto using latitude and longitude values
map_all = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_all

Extract data with Borough containing 'Toronto'

In [31]:
neighborhoods

,Borough,Neighbourhood,Latitude,Longitude
0,Scarborough Toronto,"Rouge Toronto,Malvern Toronto",43.806686,-79.194353
1,Scarborough Toronto,"Highland Creek (Toronto),Rouge Hill,Port Union...",43.784535,-79.160497
2,Scarborough Toronto,"Guildwood,Morningside Toronto,West Hill Toronto",43.763573,-79.188711
3,Scarborough Toronto,Woburn Toronto,43.770992,-79.216917
4,Scarborough Toronto,Cedarbrae,43.773136,-79.239476
5,Scarborough Toronto,Scarborough Village,43.744734,-79.239476
6,Scarborough Toronto,"East Birchmount Park,Ionview,Kennedy Park Toronto",43.727929,-79.262029
7,Scarborough Toronto,"Clairlea,Golden Mile Toronto,Oakridge Toronto",43.711112,-79.284577
8,Scarborough Toronto,"Cliffcrest,Cliffside Toronto,Scarborough Villa...",43.716316,-79.239476
9,Scarborough Toronto,"Birch Cliff,Cliffside West",43.692657,-79.264848


Train the knn model based on the location data:

In [32]:
kclusters = 3
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhoods[['Latitude','Longitude']])

In [33]:
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [72]:
import requests
import time
def get_nearby_venues(search_query, radius, lat, lng):
    
    start_time = int(time.time()) * 1000
    
    CLIENT_ID = 'DXLGSGG5AJGFKMSKWLK2QSVNWEVVI1X4VGUOUTI5DKWAKRHE' # your Foursquare ID
    CLIENT_SECRET = 'F0WEZN5V1SRNUOEKFHDT153UOPUBVIZCGTSKY1XG4GV0WGQH' # your Foursquare Secret
    VERSION = '20191219'
    LIMIT = 30
    #print('Your credentails:')
    #print('CLIENT_ID: ' + CLIENT_ID)
    #print('CLIENT_SECRET:' + CLIENT_SECRET)

    #print(search_query + ' .... OK!')
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat, lng, VERSION, search_query, radius, LIMIT)
    #print(url)
    
    results = requests.get(url).json()
    #print(results)
    # assign relevant part of JSON to venues
    venues = results['response']['venues']

    # tranform venues into a dataframe
    dataframe_test = json_normalize(venues)
    if (dataframe_test.empty):
        return None
    # keep only columns that include venue name, and anything that is associated with location
    filtered_columns = ['name', 'categories'] + [col for col in dataframe_test.columns if col.startswith('location.')] + ['id']
    dataframe_filtered = dataframe_test.loc[:, filtered_columns]

    # filter the category for each row
    dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

    # clean column names by keeping only last term
    dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

    end_time = int(time.time()) * 1000
    
    #print(str(end_time - start_time) + 'ms')
    return dataframe_filtered
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']



In [69]:
get_nearby_venues('Italian',1000, neighborhoods['Latitude'][7], neighborhoods['Longitude'][7])

https://api.foursquare.com/v2/venues/search?client_id=DXLGSGG5AJGFKMSKWLK2QSVNWEVVI1X4VGUOUTI5DKWAKRHE&client_secret=F0WEZN5V1SRNUOEKFHDT153UOPUBVIZCGTSKY1XG4GV0WGQH&ll=43.711111700000004,-79.2845772&v=20191218&query=Italian&radius=1000&limit=30
{'meta': {'code': 200, 'requestId': '5dfa24fcaba29727435deb56'}, 'response': {'venues': [{'id': '5207972e11d28a4dc2aef15a', 'name': 'Italian Espresso Bar', 'location': {'lat': 43.71570101363032, 'lng': -79.28180029795901, 'labeledLatLngs': [{'label': 'display', 'lat': 43.71570101363032, 'lng': -79.28180029795901}], 'distance': 557, 'cc': 'CA', 'country': 'Canada', 'formattedAddress': ['Canada']}, 'categories': [{'id': '4bf58dd8d48988d16d941735', 'name': 'Café', 'pluralName': 'Cafés', 'shortName': 'Café', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_', 'suffix': '.png'}, 'primary': True}], 'referralId': 'v-1576674720', 'hasPerk': False}]}}
2000ms


,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,id
0,Italian Espresso Bar,Café,43.715701,-79.2818,"[{'label': 'display', 'lat': 43.71570101363032...",557,CA,Canada,[Canada],5207972e11d28a4dc2aef15a


,name,categories,lat,lng,labeledLatLngs,distance,cc,country,formattedAddress,address,postalCode,city,state,id


In [94]:
dataframe_all_result = pd.DataFrame(columns=dataframe_all_result.columns)
dataframe_all_result
for index, row in neighborhoods.iterrows():
    dataframe_all_result = dataframe_all_result.append(
        get_nearby_venues('Chinese',2000, row['Latitude'], row['Longitude'])
    )
    print(str(index) + ':' + row['Neighbourhood'] + ' ' + str(dataframe_all_result.shape))
dataframe_all_result = dataframe_all_result.drop_duplicates(["id"],keep="last").reset_index(drop=True)
dataframe_all_result.shape

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


0:Rouge Toronto,Malvern Toronto (2, 16)
1:Highland Creek (Toronto),Rouge Hill,Port Union Toronto (2, 16)
2:Guildwood,Morningside Toronto,West Hill Toronto (4, 16)
3:Woburn Toronto (8, 16)
4:Cedarbrae (16, 16)
5:Scarborough Village (23, 16)
6:East Birchmount Park,Ionview,Kennedy Park Toronto (29, 16)
7:Clairlea,Golden Mile Toronto,Oakridge Toronto (31, 16)
8:Cliffcrest,Cliffside Toronto,Scarborough Village West (35, 16)
9:Birch Cliff,Cliffside West (39, 16)
10:Dorset Park,Scarborough Town Centre,Wexford Heights (45, 16)
11:Maryvale Toronto,Wexford Toronto (52, 16)
12:Agincourt Toronto (80, 16)
13:Clarks Corners,Sullivan,Tam O'Shanter – Sullivan (96, 17)
14:Agincourt North,L'Amoreaux East,Milliken Ontario,Steeles East (126, 17)
15:L'Amoreaux West (144, 17)
16:Upper Rouge (144, 17)
17:Hillcrest Village (153, 17)
18:Fairview,Henry Farm,Oriole (159, 17)
19:Bayview Village (166, 17)
20:Silver Hills,York Mills (171, 17)
21:Newtonbrook,Willowdale Toronto (180, 17)
22:Willowdale South (191, 17)

(233, 17)

(83, 16)

In [118]:
dataframe_all_result.sample(10)

,address,categories,cc,city,country,crossStreet,distance,formattedAddress,id,index,labeledLatLngs,lat,lng,name,neighborhood,postalCode,state
17,2873 lawrence avenue east,Chinese Restaurant,CA,Toronto,Canada,NaN,1647,"[2873 lawrence avenue east, Toronto ON, Canada]",4c674ced7abde21e94536668,NaN,"[{'label': 'display', 'lat': 43.75350728078251...",43.753507,-79.253542,Ding Ho Chinese Food,NaN,NaN,ON
27,4400 Brimley Road,Chinese Restaurant,CA,Toronto,Canada,NaN,988,"[4400 Brimley Road, Toronto ON, Canada]",52e2afb0498e24a2f325a2d2,NaN,"[{'label': 'display', 'lat': 43.78734588623047...",43.787346,-79.269852,Yummy Chinese Cuisine,NaN,NaN,ON
3,4379 Kingston Road,None,CA,Toronto,Canada,NaN,2211,"[4379 Kingston Road, Toronto ON, Canada]",4b7054e3f964a5204d132de3,NaN,"[{'label': 'display', 'lat': 43.76535407, 'lng...",43.765354,-79.190536,Wonder Season Chinese Restaurant,NaN,NaN,ON
104,4750 Yonge St.,Chinese Restaurant,CA,Toronto,Canada,Unit 2103,1281,"[4750 Yonge St. (Unit 2103), Toronto ON M2N 5M...",58c38b0dcc05d12d5e823ee3,NaN,"[{'label': 'display', 'lat': 43.76023873581672...",43.760239,-79.412161,Eupin Chinese Takeout,NaN,M2N 5M6,ON
1,NaN,Chinese Restaurant,CA,Toronto,Canada,NaN,2252,"[Toronto ON, Canada]",5345f39a498e016dd2d7ebf2,NaN,"[{'label': 'display', 'lat': 43.80614897364144...",43.806149,-79.222384,Panda Chinese,NaN,NaN,ON
26,NaN,Church,CA,NaN,Canada,NaN,702,[Canada],4b92b1f4f964a5204a1234e3,NaN,"[{'label': 'display', 'lat': 43.79755564309524...",43.797556,-79.269427,Toronto Simpson Chinese Alliance Church,NaN,NaN,NaN
160,NaN,Chinese Restaurant,CA,NaN,Canada,NaN,2159,[Canada],4c3a5d9e1e06d13a1d437b3e,NaN,"[{'label': 'display', 'lat': 43.699886, 'lng':...",43.699886,-79.431286,Tasty Chinese Restaurant,NaN,NaN,NaN
224,555 Burnhamthorpe Rd,Medical Center,CA,Toronto,Canada,West Mall,824,"[555 Burnhamthorpe Rd (West Mall), Toronto ON ...",50f05114e4b022eb98031561,NaN,"[{'label': 'display', 'lat': 43.64466980187479...",43.644670,-79.567085,Chinese Medicine & Acupuncture Academy of Toronto,NaN,M9C 2Y3,ON
71,2900 Warden Ave,Chinese Restaurant,CA,Toronto,Canada,at Finch Ave. E,72,"[2900 Warden Ave (at Finch Ave. E), Toronto ON...",4df28ea0e4cda09e6da0129a,NaN,"[{'label': 'display', 'lat': 43.79887879258761...",43.798879,-79.318335,Mr Congee Chinese Cuisine 龍粥記,L'Amoreaux,M1W 2S8,ON
139,NaN,Food,CA,NaN,Canada,NaN,828,[Canada],4e502487e4cdc76dc9bbd27b,NaN,"[{'label': 'display', 'lat': 43.686277, 'lng':...",43.686277,-79.347757,red rose chinese food,NaN,NaN,NaN


In [96]:
# create map of Toronto using latitude and longitude values
map_result = folium.Map(location=[dataframe_all_result['lat'].mean(), dataframe_all_result['lng'].mean()], zoom_start=10)

# add markers to map
for lat, lng, name, categories in zip(dataframe_all_result['lat'], dataframe_all_result['lng'], dataframe_all_result['name'], dataframe_all_result['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_result)  
    
map_result

In [103]:
kclusters = 10
kmeans = train_cluster(dataframe_all_result[['lat','lng']])

training cluster
2 Means SSE loss = 1.1083134317413412
3 Means SSE loss = 0.726358024179982
4 Means SSE loss = 0.49625172508962845
5 Means SSE loss = 0.39424442484015
6 Means SSE loss = 0.31699580913683956
7 Means SSE loss = 0.25436643039021023
8 Means SSE loss = 0.2080422697374854
9 Means SSE loss = 0.18372707716662084
10 Means SSE loss = 0.1582331833006166
11 Means SSE loss = 0.1423096273040526
12 Means SSE loss = 0.1335644359014039
13 Means SSE loss = 0.12102862926242759
14 Means SSE loss = 0.10911121664272232
15 Means SSE loss = 0.09952508315601594
16 Means SSE loss = 0.09435780916875317
17 Means SSE loss = 0.0804027311339511
18 Means SSE loss = 0.07753090947729109
19 Means SSE loss = 0.07029521163847845


In [104]:
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1,
       1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

In [102]:
import multiprocessing
def train_cluster(train_vecs, model_name=None, start_k=2, end_k=20):
    print('training cluster')
    SSE = []
    SSE_d1 = [] #sse的一阶导数
    SSE_d2 = [] #sse的二阶导数
    models = [] #保存每次的模型
    for i in range(start_k, end_k):
        kmeans_model = KMeans(n_clusters=i, n_jobs=multiprocessing.cpu_count(), )
        kmeans_model.fit(train_vecs)
        SSE.append(kmeans_model.inertia_)  # 保存每一个k值的SSE值
        print('{} Means SSE loss = {}'.format(i, kmeans_model.inertia_))
        models.append(kmeans_model)
    # 求二阶导数，通过sse方法计算最佳k值
    SSE_length = len(SSE)
    for i in range(1, SSE_length):
        SSE_d1.append((SSE[i - 1] - SSE[i]) / 2)
    for i in range(1, len(SSE_d1) - 1):
        SSE_d2.append((SSE_d1[i - 1] - SSE_d1[i]) / 2)

    best_model = models[SSE_d2.index(max(SSE_d2)) + 1]
    return best_model

In [117]:
# create map
map_clusters = folium.Map(location=[dataframe_all_result['lat'].mean(), dataframe_all_result['lng'].mean()], zoom_start=9.5)

# set color scheme for the clusters
kclusters = kmeans.n_clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, cluster in zip(dataframe_all_result['lat'], dataframe_all_result['lng'], kmeans.labels_):
    label = folium.Popup(str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=1,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
nummm = 0
for lat,lng in kmeans.cluster_centers_:
    folium.CircleMarker(
            [lat, lng],
            radius=5,
            color=rainbow[nummm-1],
            fill=True,
            fill_color=rainbow[nummm-1],
            fill_opacity=0.7).add_to(map_clusters)
    nummm = nummm + 1
       
map_clusters

In [145]:
dataframe_target_result = pd.DataFrame(columns=dataframe_all_result.columns)

for lat,lng in kmeans.cluster_centers_:
    dataframe_target_result = dataframe_target_result.append(
        get_nearby_venues('Hotel',1700, lat, lng),sort=False
    )
    print(str(dataframe_target_result.shape))
dataframe_target_result[['name','address','distance']].sort_values(by='distance').reset_index(drop=True)

(1, 17)
(14, 17)
(15, 17)


,name,address,distance
0,Delta Hotels by Marriott Toronto East,2035 Kennedy Road,1032
1,Days Hotel & Conference Centre,1677 Wilson Ave,1350
2,Hotel 89 Yorkville,89 Avenue Rd,1742
3,Howard Johnson Hotel Yorkville,89 avenue rd,1748
4,Fitness Centre,Four Seasons,1799
5,Four Seasons Hotel Toronto,60 Yorkville Avenue,1827
6,The Spa at Four Seasons Hotel Toronto,60 Yorkville Ave,1828
7,dbar,60 Yorkville Ave,1855
8,Marriot springhill hotel,612 applewood drive,1865
9,ONE Restaurant/Lounge,116 Yorkville Ave,1908


In [148]:
map_hotels = folium.Map(location=[dataframe_target_result['lat'].mean(), dataframe_target_result['lng'].mean()], zoom_start=12)
nummm = 0
colors_array_hotel = cm.rainbow(np.linspace(0, 1, len(dataframe_target_result)))
rainbow_hotel = [colors.rgb2hex(i) for i in colors_array_hotel]
for lat, lon, name in zip(dataframe_target_result['lat'], dataframe_target_result['lng'], dataframe_target_result['name']):
    label = folium.Popup(str(name), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color=rainbow_hotel[nummm - 1],
        fill=True,
        fill_color=rainbow_hotel[nummm - 1],
        fill_opacity=0.7).add_to(map_hotels)
    nummm = nummm + 1
map_hotels


In [150]:
dataframe_target_result.head()

,address,categories,cc,city,country,crossStreet,distance,formattedAddress,id,index,labeledLatLngs,lat,lng,name,neighborhood,postalCode,state
0,2035 Kennedy Road,Hotel,CA,Toronto,Canada,at Village Green Sq,1032,"[2035 Kennedy Road (at Village Green Sq), Toro...",4ad4c05cf964a520c6f520e3,NaN,"[{'label': 'display', 'lat': 43.7770701, 'lng'...",43.777070,-79.282822,Delta Hotels by Marriott Toronto East,NaN,M1T 3G2,ON
0,60 Yorkville Avenue,Hotel,CA,Toronto,Canada,btwn Bay & Yonge,1827,"[60 Yorkville Avenue (btwn Bay & Yonge), Toron...",4ad4c05bf964a520b0f520e3,NaN,"[{'label': 'display', 'lat': 43.67179643581682...",43.671796,-79.389457,Four Seasons Hotel Toronto,NaN,M4W 0A4,ON
1,154 Cumberland Street,Hotel,CA,Toronto,Canada,NaN,1997,"[154 Cumberland Street, Toronto ON M5R 1A8, Ca...",4ae1cb55f964a520758721e3,NaN,"[{'label': 'display', 'lat': 43.670009, 'lng':...",43.670009,-79.393281,Toronto Marriott Bloor Yorkville Hotel,NaN,M5R 1A8,ON
2,89 avenue rd,Hotel,CA,Toronto,Canada,NaN,1748,"[89 avenue rd, Toronto ON M5R 2G3, Canada]",4afc2c63f964a520472022e3,NaN,"[{'label': 'display', 'lat': 43.67230289047174...",43.672303,-79.395441,Howard Johnson Hotel Yorkville,NaN,M5R 2G3,ON
3,21 Avenue,Pool,CA,Toronto,Canada,NaN,2004,"[21 Avenue, Toronto ON, Canada]",4c7d1531f56a3704f9aa0946,NaN,"[{'label': 'display', 'lat': 43.66994020503226...",43.669940,-79.393417,Four Seasons Hotel Pool Deck,NaN,NaN,ON


In [175]:
venue_id = '3fd66200f964a520f4e41ee3' # ID of Harry's Italian Pizza Bar

get_venue_ratings(venue_id)

7.9


In [181]:

def get_venue_ratings(venue_id):
    
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

    results = requests.get(url).json()
    #print(results)
    # assign relevant part of JSON to venues
    venue = results['response']['venue']
    #print(venue['rating'])
    # tranform venues into a dataframe
    try:
        rating = venue['rating']
        return rating
    except:
        return None
    end_time = int(time.time()) * 1000

    #print(str(end_time - start_time) + 'ms')
    return None

In [182]:
dataframe_target_result['rating'] = None
ratings = []
for index, row in dataframe_target_result.iterrows():
    ratings.append(get_venue_ratings(row['id']))
ratings
dataframe_target_result['rating'] = ratings

[6.7,
 8.7,
 6.5,
 4.8,
 None,
 None,
 8.1,
 None,
 None,
 None,
 8.2,
 None,
 8.3,
 7.5,
 None]

In [186]:
dataframe_target_result.head()

,address,categories,cc,city,country,crossStreet,distance,formattedAddress,id,index,labeledLatLngs,lat,lng,name,neighborhood,postalCode,state,rating
0,2035 Kennedy Road,Hotel,CA,Toronto,Canada,at Village Green Sq,1032,"[2035 Kennedy Road (at Village Green Sq), Toro...",4ad4c05cf964a520c6f520e3,NaN,"[{'label': 'display', 'lat': 43.7770701, 'lng'...",43.777070,-79.282822,Delta Hotels by Marriott Toronto East,NaN,M1T 3G2,ON,6.7
0,60 Yorkville Avenue,Hotel,CA,Toronto,Canada,btwn Bay & Yonge,1827,"[60 Yorkville Avenue (btwn Bay & Yonge), Toron...",4ad4c05bf964a520b0f520e3,NaN,"[{'label': 'display', 'lat': 43.67179643581682...",43.671796,-79.389457,Four Seasons Hotel Toronto,NaN,M4W 0A4,ON,8.7
1,154 Cumberland Street,Hotel,CA,Toronto,Canada,NaN,1997,"[154 Cumberland Street, Toronto ON M5R 1A8, Ca...",4ae1cb55f964a520758721e3,NaN,"[{'label': 'display', 'lat': 43.670009, 'lng':...",43.670009,-79.393281,Toronto Marriott Bloor Yorkville Hotel,NaN,M5R 1A8,ON,6.5
2,89 avenue rd,Hotel,CA,Toronto,Canada,NaN,1748,"[89 avenue rd, Toronto ON M5R 2G3, Canada]",4afc2c63f964a520472022e3,NaN,"[{'label': 'display', 'lat': 43.67230289047174...",43.672303,-79.395441,Howard Johnson Hotel Yorkville,NaN,M5R 2G3,ON,4.8
3,21 Avenue,Pool,CA,Toronto,Canada,NaN,2004,"[21 Avenue, Toronto ON, Canada]",4c7d1531f56a3704f9aa0946,NaN,"[{'label': 'display', 'lat': 43.66994020503226...",43.669940,-79.393417,Four Seasons Hotel Pool Deck,NaN,NaN,ON,NaN


In [189]:
dataframe_target_result.sort_values(by=['rating'],ascending=False)[['name','address','distance','rating']].head(5)

,name,address,distance,rating
0,Four Seasons Hotel Toronto,60 Yorkville Avenue,1827,8.7
11,dbar,60 Yorkville Ave,1855,8.3
9,The Hazelton Hotel,118 Yorkville Avenue,1913,8.2
5,The Spa at Four Seasons Hotel Toronto,60 Yorkville Ave,1828,8.1
12,ONE Restaurant/Lounge,116 Yorkville Ave,1908,7.5
